# <center>REVIEW RATING PREDICTION</center>


### Problem Statement

We have a client who has a website where people write different reviews for technical products. Now they are adding a new feature to their website i.e. The reviewer will have to add stars(rating) as well with the review. The rating is out 5 stars and it only has 5 options available 1 star, 2 stars, 3 stars, 4 stars, 5 stars. Now they want to predict ratings for the reviews which were written in the past and they don’t have a rating. So, we have to build an application which can predict the rating by seeing the review.

First we scrape 20000 reviews of product(Mobile phones) from flipkart.com .then now we will make model for it.


IMPORT LIBRARIES AND LOAD DATASET

In [ ]:
##Importing all necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('final-comment-scraping.csv')
df.head()

,Unnamed: 0,Rating,Comment
0,0,4,🤩🤩🤩 I am very very happy😊😊
1,1,4,👏👏 Delivery service is good 👍\nThis phone is a...
2,2,5,👌👌good control
3,3,4,👌👌👌Very good product I am very happy 👌👌👌 😊😊😊😊😊😊
4,4,5,💫✨Let it be✨


In [ ]:
df.shape

(23970, 3)

IN traning set we have 23970 rows or comment and 3 variables 

In [ ]:
df.info()

#2 columns is objective(id and comment_text) and other is int form of 0 or 1 where 0 is not toxic and 1 means malignant.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23970 entries, 0 to 23969
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  23970 non-null  int64 
 1   Rating      23970 non-null  int64 
 2   Comment     23970 non-null  object
dtypes: int64(2), object(1)
memory usage: 561.9+ KB


In [ ]:
df.describe()

# as per mean it looking skewed data 

,Unnamed: 0,Rating
count,23970.000000,23970.000000
mean,4561.181519,4.529412
std,3040.325269,0.687035
min,0.000000,3.000000
25%,1997.000000,4.000000
50%,3994.500000,5.000000
75%,6950.000000,5.000000
max,11393.000000,5.000000


In [ ]:
df = df.drop(['Unnamed: 0'], axis = 1)

In [ ]:
pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-k795bsq5
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-k795bsq5
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=11756 sha256=95632af29236e5e6541bd8c7cde4bb44faa6556a0c6310679d30116c6e2cfb96
  Stored in directory: /tmp/pip-ephem-wheel-cache-t03gutv4/wheels/0d/b3/29/bfe3deffda68980088d17b81331be6667e837ffb4a071bae82
Successfully built preprocess-kgptalkie


In [ ]:
import preprocess_kgptalkie as ps
import re

In [ ]:
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [ ]:
df['Comment'] = df['Comment'].apply(lambda x: get_clean(x))

In [ ]:
df

,Rating,Comment
0,4,i am very very happy
1,4,delivery service is good this phone is a kille...
2,5,good control
3,4,very good product i am very happy
4,5,let it be
...,...,...
23965,5,best phone go ahead to buy it
23966,5,really nice product
23967,5,really a great phone from motorola
23968,5,gets heated a lot camera choppy does not have ...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


In [ ]:
tfidf = TfidfVectorizer(max_features =20000,ngram_range =(1,5), analyzer='char')

In [ ]:
X = tfidf.fit_transform(df['Comment'])
y = df['Rating']

In [ ]:
X.shape, y.shape

((23970, 20000), (23970,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 0)

In [ ]:
X_train.shape

(19176, 20000)

In [ ]:
clf = LinearSVC(C = 5, class_weight = 'balanced')
clf.fit(X_train, y_train)

LinearSVC(C=5, class_weight='balanced')

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           3       0.12      0.09      0.10       541
           4       0.25      0.14      0.18      1173
           5       0.65      0.79      0.72      3080

    accuracy                           0.55      4794
   macro avg       0.34      0.34      0.33      4794
weighted avg       0.49      0.55      0.52      4794



In [ ]:
x = 'this Product is really awesome, i like it'
x = get_clean(x)
vec = tfidf.transform([x])
clf.predict(vec)

array([5])